Implemented on top of AzureML.
Reference: https://docs.microsoft.com/en-us/azure/machine-learning/

Pipelines are going to be created inside some Azure ML workspace.
If you already have one, you can get the reference:

In [1]:
# if you already have a workspace created, just run this to initialize workspace config
# if you haven't created a workspace, skip this
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: C:\dev\reinforcement_learning\batch\aml_config\config.json


In case if you do not have, any you can either create it on Azure portal or programmatically:

In [ ]:
# if you don't have a workspace already created then run this step
from azureml.core import Workspace
ws = Workspace.create(name = "cheng_offline_dashboard",
                      subscription_id = "9a2d7383-3c7d-492c-94fc-ba65be408672",
                      resource_group = "cps-dev-cheng",
                      create_resource_group = False,
                      location = 'eastus'
                     )
ws.write_config()
ws.get_details()

This is a library of pipeline steps / primitives for creating batch training pipelines from logs produced by personalization service. Also, you can find some recipe pipelines which can be either used as-is or as a reference.

For creating pipeline application context is required:

In [ ]:
ws.datastores # list out all datastores

In [2]:
import application
from application import context

# set up where to read the log files
ctx = context.Context(
    accountName = 'chengmystorage',
    accountKey = 'OOvXDYUpXqfd9kyipcGyThuVQqh+avvHl36hRcBjeA53FBBtLkx0L0dRJhqpMJV4dXVfYGOWngH9qols20rzPQ==',
    appId = 'large',
    appFolder = 'folder'
)

Now you can create batch training pipeline for vw batch training:

Created pipeline can be either submitted for execution:

Or published to be available via REST endpoint:

Example of REST call:
Reference for better understanding of Azure ML authentication:
https://github.com/Azure/MachineLearningNotebooks/blob/f16bf27e26d6ff7a2cf76e8cbcda4ccc9c878a0c/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb

In [3]:
from pipelines import extract_pipeline
extractPipeline = extract_pipeline.create_pipeline(ws, ctx, parallel_jobs = 2) # I only parallel running 2 jobs for now for debugging purpose

Found existing cluster: python
Best command selection step is created successfully
Data store for the app context {chengmystoragelarge} is found
Found existing cluster: extractor
Cache step is successfully created
Found existing cluster: vw
Found existing cluster: python
Vw sweep step is successfully created
Found existing cluster: vw-predict
Vw predict step is successfully created
Data store for the app context {chengmystoragelarge} is found
Found existing cluster: dashboard
Dashboard step is successfully created
extractPipeline is succesfully created.
Step Dashboard is ready to be created [38ee7727]
extractPipeline is succesfully validated.


In [4]:
# pipeline params set up the date range of logs

from azureml.core import Experiment  
regen_outputs = False
extractExperiment = Experiment(ws, 'test_sweep').submit(
    extractPipeline, 
    pipeline_params = {
        "start_date":'10/21/2018',
        "end_date":'10/22/2018'
    }, 
    regenerate_outputs = regen_outputs
)
print("Extract Pipeline is submitted for execution")
from azureml.widgets import RunDetails
RunDetails(extractExperiment).show()

Created step Prepare command [9be3bbbb][1a2c417c-2a6e-49ce-a936-52aecb8ba997], (This step is eligible to reuse a previous run's output)
Created step Cache [71be217b][85e7dcd0-683b-435e-bc53-c4eed258a880], (This step is eligible to reuse a previous run's output)
Created step Predict [6859dd19][e0be848c-c1fd-4128-bc86-7cadce3ee099], (This step is eligible to reuse a previous run's output)
Created step Best command [cccdb208][738d1093-8e6f-432a-97df-597b19220fcb], (This step is eligible to reuse a previous run's output)
Created step Sweep [vw] [2f498876][72e0c6d3-6cbc-4b02-bf50-bb0a281a0f07], (This step will run and generate new outputs)
Created step Dashboard [38ee7727][73efb002-5383-45fb-afc2-2cb21bd1adbf], (This step will run and generate new outputs)
Using data reference Application_logs for StepId [223e57f1][2dd58e67-bc3a-4b0f-b8c2-126ee8484f4f], (Consumers of this data are eligible to reuse prior runs.)
Using data reference Application_logs for StepId [f1ace1c9][2dd58e67-bc3a-4b0f-b

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …